In [1]:
import pandas as pd
from scipy.optimize import curve_fit
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
from datetime import datetime, date

C:\Users\etiirine\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Reading in the data

In [2]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,12,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[1]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

#time["Elapsed time, mass 40"].describe()

       mass 2.0_value  mass 4.0_value  mass 12.0_value  mass 16.0_value  \
count    6.400000e+02    6.400000e+02     6.400000e+02     6.400000e+02   
mean     1.239396e-10    6.194648e-13     1.331821e-12     2.341213e-12   
std      4.632134e-12    7.739677e-14     9.556464e-13     4.507586e-13   
min      1.029900e-10    4.000000e-16     5.990000e-13     1.237000e-12   
25%      1.221140e-10    5.843250e-13     1.135600e-12     2.272275e-12   
50%      1.233235e-10    6.162500e-13     1.195950e-12     2.306550e-12   
75%      1.260110e-10    6.520250e-13     1.324275e-12     2.346450e-12   
max      1.435500e-10    1.130700e-12     1.206840e-11     1.161800e-11   

       mass 18.0_value  mass 28.0_value  mass 32.0_value  mass 40.0_value  \
count     6.400000e+02     6.400000e+02     6.400000e+02     5.000000e+00   
mean      5.740230e-12     3.143339e-13     1.732423e-12     1.763240e-12   
std       4.227418e-13     4.341662e-13     3.855449e-13     8.668836e-13   
min       2.3756

mass 2.0_time            mass 4.0_time  \
1    2022/12/01 17:03:07.315  2022/12/01 17:03:07.565   
2    2022/12/01 17:03:09.638  2022/12/01 17:03:13.139   
3    2022/12/01 17:03:15.462  2022/12/01 17:03:18.963   
4    2022/12/01 17:03:21.286  2022/12/01 17:03:24.787   
5    2022/12/01 17:03:27.110  2022/12/01 17:03:30.611   
..                       ...                      ...   
636  2022/12/01 18:04:42.685  2022/12/01 18:04:46.186   
637  2022/12/01 18:04:48.510  2022/12/01 18:04:52.011   
638  2022/12/01 18:04:54.335  2022/12/01 18:04:57.836   
639  2022/12/01 18:05:00.160  2022/12/01 18:05:03.661   
640  2022/12/01 18:05:05.985  2022/12/01 18:05:09.486   

              mass 12.0_time           mass 16.0_time  \
1    2022/12/01 17:03:07.815  2022/12/01 17:03:08.065   
2    2022/12/01 17:03:13.389  2022/12/01 17:03:13.640   
3    2022/12/01 17:03:19.213  2022/12/01 17:03:19.463   
4    2022/12/01 17:03:25.037  2022/12/01 17:03:25.287   
5    2022/12/01 17:03:30.861  2022/12/01 17:03:31.112   
..                       ...                      ...   
636  2022/12/01 18:04:46.436  2022/12/01 18:04:46.687   
637  2022/12/01 18:04:52.261  2022/12/01 18:04:52.512   
638  2022/12/01 18:04:58.086  2022/12/01 18:04:58.337   
639  2022/12/01 18:05:03.911  2022/12/01 18:05:04.162   
640  2022/12/01 18:05:09.737  2022/12/01 18:05:09.987   

              mass 18.0_time           mass 28.0_time  \
1    2022/12/01 17:03:08.315  2022/12/01 17:03:08.566   
2    2022/12/01 17:03:13.890  2022/12/01 17:03:14.140   
3    2022/12/01 17:03:19.713  2022/12/01 17:03:19.963   
4    2022/12/01 17:03:25.537  2022/12/01 17:03:25.788   
5    2022/12/01 17:03:31.362  2022/12/01 17:03:31.612   
..                       ...                      ...   
636  2022/12/01 18:04:46.937  2022/12/01 18:04:47.187   
637  2022/12/01 18:04:52.762  2022/12/01 18:04:53.012   
638  2022/12/01 18:04:58.587  2022/12/01 18:04:58.837   
639  2022/12/01 18:05:04.412  2022/12/01 18:05:04.662   
640  2022/12/01 18:05:10.237  2022/12/01 18:05:10.487   

              mass 32.0_time           mass 40.0_time  \
1    2022/12/01 17:03:08.816  2022/12/01 17:03:09.066   
2    2022/12/01 17:03:14.640                      NaN   
3    2022/12/01 17:03:20.464                      NaN   
4    2022/12/01 17:03:26.038  2022/12/01 17:03:26.288   
5    2022/12/01 17:03:32.112                      NaN   
..                       ...                      ...   
636  2022/12/01 18:04:47.688                      NaN   
637  2022/12/01 18:04:53.512                      NaN   
638  2022/12/01 18:04:59.337                      NaN   
639  2022/12/01 18:05:05.162                      NaN   
640  2022/12/01 18:05:10.987                      NaN   

              mass 44.0_time  Elapsed time, mass 2  Elapsed time, mass 4  \
1    2022/12/01 17:03:09.317                 0.000                 0.000   
2                        NaN                 2.323                 5.574   
3                        NaN                 8.147                11.398   
4                        NaN                13.971                17.222   
5                        NaN                19.795                23.046   
..                       ...                   ...                   ...   
636                      NaN              3695.370              3698.621   
637                      NaN              3701.195              3704.446   
638                      NaN              3707.020              3710.271   
639                      NaN              3712.845              3716.096   
640                      NaN              3718.670              3721.921   

     Elapsed time, mass 12  Elapsed time, mass 16  Elapsed time, mass 18  \
1                    0.000                  0.000                  0.000   
2                    5.574                  5.575                  5.575   
3                   11.398                 11.398                 11.398   
4                   17.222                 17.222         

In [3]:
#reading the vaclogger software file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

Live comments                 Time  Elapsed time  injection 100mbar  \
1           NaN  01/12/2022 17:02:31           7.0            0.45540   
2           NaN  01/12/2022 17:02:38          14.0            0.45555   
3           NaN  01/12/2022 17:02:45          21.0            0.45466   
4           NaN  01/12/2022 17:02:53          29.0            0.45522   
5           NaN  01/12/2022 17:03:00          36.0            0.45537   

       Barion_2      Barion_1  DUAL experiment  DUAL insulation  \
1  4.330000e-09  4.890000e-09     4.996000e-09     3.934000e-07   
2  4.320000e-09  4.890000e-09     4.996000e-09     3.939000e-07   
3  4.330000e-09  4.900000e-09     4.996000e-09     3.939000e-07   
4  4.320000e-09  4.900000e-09     4.996000e-09     3.939000e-07   
5  4.320000e-09  4.890000e-09     4.996000e-09     3.939000e-07   

   injection 1mbar  helium  T-platinum  T-CERNOX  I_emission  I_grid  
1         0.000583   246.0      -5.163  4296.279         NaN     NaN  
2         0.000582   245.0      -5.050  4296.279         NaN     NaN  
3         0.000584   245.0      -5.163  4296.167         NaN     NaN  
4         0.000570   246.0      -5.163  4296.167         NaN     NaN  
5         0.000582   245.0      -5.163  4296.167         NaN     NaN

In [4]:
#Reading in the HiVolta measurement file
hv = pd.read_csv("hv log", sep=",")
#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["em"]=[element * 1e-6 for element in hv["IMon2"]]
hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["em"]=abs(hv["em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding an elapsed time column
timestamp=[]
for i in hv["Time"]:
    timestamp.append(i)
    
#making the column into a datetime object
timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)
hv.drop(index=hv.index[0], 
        axis=0, 
        inplace=True)

hv.describe()



Elapsed time        VMon1        VMon2        VMon3        VMon4  \
count   3040.000000  3040.000000  3040.000000  3040.000000  3040.000000   
mean    1813.778715   223.875099    90.787566   906.609697   906.385447   
std     1048.004185   248.084453    29.032126   289.468562   289.853747   
min        1.183000     0.020000     0.020000     0.000000     0.000000   
25%      906.230500     0.100000   100.080000   999.880000  1000.000000   
50%     1813.731000     0.140000   100.100000   999.880000  1000.000000   
75%     2720.638500   499.900000   100.100000   999.880000  1000.000000   
max     3639.191000   499.900000   100.160000   999.940000  1000.040000   

             VMon5   VMon6   VMon7        VMon8        IMon1        IMon2  \
count  3040.000000  3040.0  3040.0  3040.000000  3040.000000  3040.000000   
mean    906.863691     0.0     0.0     0.000651    -0.197822     0.690393   
std     289.129262     0.0     0.0     0.003551     0.239700     0.763334   
min       0.000000     0.0     0.0     0.000000    -0.675400    -3.132600   
25%    1000.000000     0.0     0.0     0.000000    -0.420725     0.017000   
50%    1000.000000     0.0     0.0     0.000000    -0.002400     0.019200   
75%    1000.020000     0.0     0.0     0.000000     0.016000     1.457250   
max    1000.180000     0.0     0.0     0.020000     0.544400     3.159900   

             IMon3        IMon4        IMon5        IMon6        IMon7  \
count  3040.000000  3040.000000  3040.000000  3040.000000  3040.000000   
mean     13.261068    13.287406    13.279344    -0.018159    -0.030587   
std       4.361207     4.368394     4.366782     0.001506     0.001157   
min      -0.002100     0.005900    -0.041600    -0.021300    -0.033600   
25%      13.455175    13.505925    13.479700    -0.019400    -0.031500   
50%      14.626000    14.657400    14.646200    -0.017600    -0.030600   
75%      15.443225    15.470750    15.465025    -0.017000    -0.029500   
max      16.372600    16.401500    16.356900    -0.016100    -0.028600   

             IMon8       hv_grid            em  
count  3040.000000  3.040000e+03  3.040000e+03  
mean     -0.017762  2.151142e-07  7.034527e-07  
std       0.000834  2.243066e-07  7.513114e-07  
min      -0.019900  7.000000e-10  6.700000e-09  
25%      -0.018300  1.540000e-08  1.700000e-08  
50%      -0.018000  1.720000e-08  1.920000e-08  
75%      -0.017000  4.210000e-07  1.458325e-06  
max      -0.016000  6.754000e-07  3.159900e-06

### Data processing

In [5]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [6]:
#Filtering background signal

#Find the timestamp where emission was turned on
print("Result 1:",hv.loc[hv["Comment"]=="grid repelling"])
x=pd.Timestamp(2022, 11, 14, 17,12, 45)
#y=pd.Timestamp(2022, 11, 10, 17,12, 45)
#then find the time value closest to it in the MID scan file
print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
#print("\n","Result 2:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])

#average the signal before this timestamp and subtract it from the Em-on part
bck=np.mean(mid["mass 40.0_value"])
print("\n","Result 3:",bck)

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck
print("Res 1:",mid["mass 2, corr"].describe())
#remove signal below 10^-14
mid["mass 32.0_value"][mid["mass 32.0_value"] < 1e-14] = np.nan

#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][102:195]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][0:102]/h2_sens
base_h2.describe()

Result 1:             Date          Time  Elapsed time   VMon1   VMon2   VMon3   VMon4  \
814   01/12/2022  17:20:27.511       970.252  499.82  100.14  999.88  1000.0   
815   01/12/2022  17:20:28.701       971.442  499.82  100.14  999.88  1000.0   
816   01/12/2022  17:20:29.896       972.637  499.82  100.12  999.88  1000.0   
817   01/12/2022  17:20:31.091       973.832  499.82  100.12  999.88  1000.0   
2472  01/12/2022  17:53:27.130      2949.871  499.82  100.12  999.88  1000.0   
2473  01/12/2022  17:53:28.319      2951.060  499.82  100.12  999.88  1000.0   
2474  01/12/2022  17:53:29.510      2952.251  499.82  100.12  999.88  1000.0   
2475  01/12/2022  17:53:30.704      2953.445  499.82  100.12  999.88  1000.0   

        VMon5  VMon6  VMon7  ...   IMon2    IMon3    IMon4    IMon5   IMon6  \
814   1000.00    0.0    0.0  ...  0.0311  13.6475  13.6160  13.7001 -0.0188   
815   1000.00    0.0    0.0  ...  0.0251  13.6427  13.6136  13.6975 -0.0188   
816   1000.00    0.0    0.0  ...

C:\Users\etiirine\AppData\Local\Temp\ipykernel_10488\765137446.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mid["mass 32.0_value"][mid["mass 32.0_value"] < 1e-14] = np.nan


count    1.020000e+02
mean     5.673060e-09
std      4.467123e-10
min      4.752430e-09
25%      5.604355e-09
50%      5.659731e-09
75%      6.059777e-09
max      6.509285e-09
Name: mass 2, corr, dtype: float64

### Plotting

In [7]:
#Plotting the all the MID channels
plt.figure(figsize=(14,8))
plt.plot(time['Elapsed time, mass 2'],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(time['Elapsed time, mass 4'],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(time['Elapsed time, mass 12'],mid["mass 12.0_value"],marker=".", markersize=4,label='M14')
plt.plot(time['Elapsed time, mass 16'],mid["mass 16.0_value"],marker=".", markersize=4,label='M16')
plt.plot(time['Elapsed time, mass 18'],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(time['Elapsed time, mass 28'],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(time['Elapsed time, mass 32'],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(time['Elapsed time, mass 40'],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
plt.plot(time["Elapsed time, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("MID scan @~RT EGA on Cu  with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(0, 7e-11, '\u2191 No emission', fontdict = font)
plt.text(400, 1.7e-10, 'Em. on, grid transparent', fontdict = font)
plt.text(1400, 8e-11, 'Grid repelling', fontdict = font)
plt.text(2300, 1.7e-10, 'Grid transparent', fontdict = font)
plt.text(3000, 7.75e-11, 'Grid transparent', fontdict = font)
plt.text(2700, 7.75e-12, 'EGA turned \u2192', fontdict = font)



plt.yscale('log')

In [8]:
#plotting HiVolta log
plt.figure(figsize=(12,6))
plt.plot(hv["Elapsed time"],hv["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(hv["Elapsed time"],hv["em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("Current evolution @~RT EGA on Cu  with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(-100, 1.15e-8, 'No emission', fontdict = font)
plt.text(435, 1.9e-6, 'Em. on, grid transparent', fontdict = font)
plt.text(1365, 2.57e-8, 'Grid repelling', fontdict = font)
plt.text(2294, 1.9e-6, 'Grid transparent', fontdict = font)
plt.text(2750, 9.1e-9, 'Grid repelling, EGA rotated', fontdict = font)

plt.yscale('log')


In [9]:
#Plotting total pressures
plt.figure(figsize=(10,6))
plt.plot(vaclog["Elapsed time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.plot(vaclog["Elapsed time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='F-R')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("Total pressure evolution @~RT EGA on Cu with pre-injection of 1 ML")
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 12,
        }
plt.text(250, 1.13e-8, '\u2190 EGA moved into position ', fontdict = font)
plt.text(250, 4.5e-9, '\u2191 No emission', fontdict = font)
plt.text(480, 6.5e-9, 'Em. on, Grid transparent', fontdict = font)
plt.text(1240, 4.5e-9, 'Grid repelling', fontdict = font)
plt.text(2360, 6.15e-9, 'Grid transparent', fontdict = font)
plt.text(2500, 1.6e-8, 'EGA rotated \u2192', fontdict = font)

ax = plt.gca()
ax.set_yscale('log')
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')
plt.show()

# Primary desorption calculation method - not applicable due to friction induced pressure spike in base pressures
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.


In [10]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 296
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current
I_em = np.mean(hv["em"].nlargest(30))
print(I_em)

2.5479115256582912e-08
0.01992516581153798
2.121243333333333e-06


In [11]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: 0.009374432413451879 

